## Coinciding Vertices
The `coincidingVertices` function is an essential component in mesh processing algorithms, designed to identify and handle vertices with the same coordinates. This functionality becomes crucial in scenarios where mesh data contains overlapping vertices due to precision limitations or data noise.

In [25]:
import numpy as np
from maniflow.mesh import Mesh, Face
from maniflow.mesh.obj import OBJFile



## The upper triangular approach
The old version of `coincidingVertices` utilizes an approach with a time complexity of $O(n^2)$, where n represents the number of vertices in the mesh. While being naive in its concept there are a few tweaks that improve the algorithm, more than halving its runtime. It starts by initializing empty data structures, iterates through each vertex to detect coinciding vertices, merges them into a single vertex, and updates the mesh accordingly. To reduce computational power needed this naive approach is improved by the simple fact that if $a == b$ then also $b == a$, so this halves the amount of computations. This leaves us with an upper triangular form where we only have to check the following indices but not the previous as those where checked before. E.g. when checking whether the third vertex can be identified with another vertex we can start at the fourth vertex since, say, the check against the second vertex already happened when we previously check for identities with the second vertex. The resulting matrix looks similar to this

<center>

| 1 | 2 | 3 | 4 | ... | n |
|---|---|---|---|---|---|
|   | 2 | 3 | 4 | ... | n |
|   |   | 3 | 4 | ... | n |
|   |   |   | 4 | ... | n |
|   |   |   |   | ... | n |
|   |   |   |   |   | n |

</center>

Its code looks the following:

In [26]:
def coincidingVertices_old(mesh: Mesh):
    """
    A method to identify vertices with the same coordinates with each other [O(V^2)].
    This way we can "clue" edges together that share the same coordinates of technically different vertices.
    The approach is similar to an upper triangle matrix as we don't need to reverse check vertices,
    e.g. v1==v2 doesn't require additional v2==v1 check.
    :param mesh: the mesh of which vertices should be 
    """
    dummy = mesh.copy()
    verts = list()
    lookup = dict()

    for i in range(mesh.v):
        # i in lookup implies that the i-th vertex is equal some previous i'-th vertex with i'<i
        # thus was taken care of as i+1+j in the following steps
        if i in lookup:
            continue
        for j in range(mesh.v-i-1):
            # taken care of in previous iteration of some smaller i
            if i+1+j in lookup:
                continue
            # vertex coordinates are same or at least very close
            if np.allclose(mesh.vertices[i], mesh.vertices[i+1+j], atol=1e-06):
                lookup[i+1+j] = len(verts)  # linking to index in new vertex list
        lookup[i] = len(verts)  # adding i to dict as i not in lookup before (see "if" above)
        verts.append(mesh.vertices[i])

    # update the faces
    dummy.vertices = verts
    dummy.faces = list({Face(mesh, *[lookup[i] for i in face.vertices]) for face in mesh.faces})
    dummy.resetFaceGraph()
    return dummy
    

### NB
Looking back, this approach is, in its core, equivalent to the bubblesort sorting algorithm. The simplified pseudocode of bubblesort with adjustments to highlight similarities proves this as it consists of the same two for loops as the upper triangle approach with an if-condition nested inside them. Bubblesort has the 'swapped'-flag to break early in case no swap occured during an iteration indicating that the elements are all sorted. A similar thing happens in the upper triangle approach with the use of the 'lookup'-list where we skip through the iteration of $i,j$ if a vertex was already coincided.

**Input:** Unsorted list 'arr' with length 'n'

**Pseudocode Bubblesort**
- For i := 1 to n - 1 do
    - swapped := False
    - For j := 1 to n - i do
        - If arr[j - 1] > arr[j] then
            - swap(arr[j - 1], arr[j])
            - swapped := True
    - If swapped then
        - Break
        

Bubblesort also first iterates through all elements. At the end of the iteration one of the elements is at its proper place. Afterwards, it repeats with the next iteration but this time it can leave out checking said element. Basically, we also build ourselves an upper triangle (reversed as bubblesort fixes the last element) when visualizing the iterations with the elements the respective iteration checks where the 'X' indicates bubblesort having compared the value at that position:

<center>

| X | X | X | X | ... | X | X |
|---|---|---|---|---|---|---|
| X | X | X | X | ... | X |   | 
| X | X | X | X | ... |  |   |
| X | X | X | X |  |  |   |
| X | X | X |   |  |  |   |
| X | X |   |   |  |  |   |
| X |   |   |   |  |  |   |

</center>

Therefore bubblesort and the upper triangle approach share the same performance with a best-case scenario of $O(n)$, where bubblesort makes use of the fact that it is adaptive, meaning that it takes advantage of the existing order and does not swap if the order already fits the sorting goal. The same is true for the upper triangle approach as when all of the points lie close within the tolerance value then after one iteration we are done with only the initial first vertex left. Also, the worst-case scenario is the same that when bubblesort has to always change the positions of the elements checked and the upper triangle approach has to work through all possible vertex pairs as there is no coinciding, leaving the two with a performance of $O(n^2)$.

## Time to sort - Using Timsort to speed things up
The updated version of `coincidingVertices` employs a more efficient approach compared to the old version. It starts by creating an empty list `extVertList` and copies the mesh's faces into `faceList`. It then iterates through each face and its vertices, adding the j-th vertex in the i-th face along with the information on i and j to `extVertList`. Next, it sorts `extVertList` by the first element (the vertex coordinates) using Python's Timsort algorithm, resulting in a stable sorting with an average time complexity of O(n log(n)).

The function then initializes an empty list `vertList` and iterates through all entries of `extVertList`. For each entry `v`, it checks if `vertList` is empty or if the vertex is not close to the previous one. If so, it adds the vertex to `vertList`. Additionally, it updates the vertices in `faceList` based on the indices in `vertList`, ensuring that vertices with the same coordinates are identified and glued together.

Note that the definition of closeness is not the standard metric but whether the vertices can be fitted into a box of size *2\*atol x 2\*atol x 2\*atol*. While not as intuitive as a simple distance between the points in the three dimensional, this allows us to use the a sorting algorithm since the vertices have to be within atol on *all* three coordiante axis. This means we can start looking at the first coordiante, compare that and, if it is within tolerance, we can continue with the second and third if needed. Hence, we can sort the first coordiantes (if equal then go by the second, etc.). This approach can be used, as we did, to implement a fast algorithm that only needs to check the previous vertex for equality since it is the closest yet tested vertex, at least on the first coordinate.

The code is the following:

In [27]:
def coincidingVertices_new(mesh: Mesh):
    """
    A method to identify vertices with the same coordinates with each other.
    This way we can "glue" edges together that share the same coordinates of technically different vertices.
    The approach is similar to an upper triangle matrix as we don't need to reverse check vertices,
    e.g. v1==v2 doesn't require additional v2==v1 check.
    :param mesh: the mesh of which vertices should be checked
    """
    dummy = mesh.copy()
    extVertList = list()
    faceList = mesh.faces.copy()

    # Iterate through all faces and their respective vertices
    # Add the j-th vertex in the i-th face together with the info on i and j to extVertList
    for i in range(mesh.f):
        for j in range(len(mesh.faces[i])):
            extVertList.append([mesh.faces[i][j], i, j])    # the j-th vertex in the i-th face
        
    # Sort by the first element (the vertex coordinates), e.g.
    # [2, 1, 3], [3, 4, 5], [2, 3, 1], [1, 3, 2]
    # results in
    # [1, 2, 3], [2, 1, 3], [2, 3, 1], [3, 4, 5]
    # Python uses Timsort as sort()-algorithm which is stable and on average takes O(n log(n)).
    extVertList.sort(key = lambda x : list(x[0]))

    # Initialize list vertList
    vertList = list()

    # Iterating through all entries of extVertList
    # This does include multiple instances of the same vertex which occurs in different faces
    for v in extVertList:
        # If either vertList is empty (first iteration) or vertex is not close to the previous one, add to the varying vertices list
        # Note that all coordinates have to be within tolerance for vertices to be seen as equal (so within a block of atol x atol x atol size)
        # Therefore we can simply start checking from the first coordinate and directly reject if too far away, otherwise continue with second coordiante
        if (not vertList) or (not np.allclose(v[0], vertList[-1], atol=1e-06)):
            vertList.append(v[0])
        # As mesh.faces.vertices is stored as tuple (cp. Face constructor) we need to temporarily change it to a list for item assignement
        face_vertices_list = list(faceList[v[1]].vertices)
        face_vertices_list[v[2]] = len(vertList) - 1
        faceList[v[1]].vertices = tuple(face_vertices_list)

    # Update the faces
    dummy.vertices = vertList
    dummy.faces = faceList
    dummy.resetFaceGraph()
    return dummy

## Time comparison
A direct comparison can be drawn by using the 'timeit'-package in Python. Since coincidingVertices normally overwrites the mesh given in the parameter we need to make adjustments to the functions. We introduce a variable called 'dummy' that is written into instead of the original mesh. It is only used for this writing of data.

The comparision will be drawn between the two implementations using the teapot OBJ-file already present. It consists of 3644 vertices which both functions will bring down to 3241. This is a decrease of about about 11.1% or one in every nine vertices being identified with another vertex.

As the main load of the computation is on the naive double-looping (upper triangle) and the sorting algorithm we can expect the other calls in the functions such as list manipulations and calls to stored data to be insignificant with the difference in performance being $O(n^2)$ and $O(n\log n)$ on average for a relatively (to simpler shapes) large $n=3644$.

In [32]:
from timeit import default_timer as timer

teapot = OBJFile.read("examples/teapot.obj")

repetitions = 10    # coincidingVertices_old is SIGNIFICANTLY slower than the newer version, choose repetitions accordingly
time_a = 0
time_b = 0

for i in range(repetitions):
    start = timer()
    coincidingVertices_old(teapot)
    end = timer()
    time_a = time_a + (end - start)

for i in range(repetitions):
    start = timer()
    coincidingVertices_new(teapot)
    end = timer()
    time_b = time_b + (end - start)

print(f"Upper Triangle Approach execution time: {time_a/repetitions} seconds")
print(f"Sorting Algorithm Approach execution time: {time_b/repetitions} seconds")

576
576
Upper Triangle Approach execution time: 2.3616827099976945 seconds
Sorting Algorithm Approach execution time: 0.03945104500235175 seconds


### Remark
The upper triangle approach needs considerably longer by a factor of 235. This is within expectation as if we assume the upper triangle to be $O(1/2 * n^2)$ (since we only use half the matrix), we get
$$\left|\frac{t_{\text{upper triangle}}}{t_{\text{sort}}}\right|=\frac{O(\frac{1}{2}\cdot n^2)}{O(n\cdot\log(n))}=O\left(\frac{n}{2\log(n)}\right)\leq M\cdot\left(\frac{n}{\log(n^2)}\right)$$
for sufficiently large $n$ and some fixed positive real $M$. For $n=3644$ (the amount of vertices of the teapot) we get $\frac{n}{2\log(n)}=222$ which means that the inequality doesn't rely (yet) on $M$ being large.

## Notes on Comparability and Downsides
With this speedy approach come limitation, though. The algorithm does not care for relative closeness, all it looks at is whether it is within atol but not *how* close the vertices are when within this tolerance. This can lead to cases where for three points $[a, b, c]$ with $a$ and $b$ being close (as in within the atol-box) and $b$ and $c$ being close, but not $a and $c$.

The algorithm starts from $a$ and checks against $b$. They are close so it identifies $a$ and $b$ with coordinates of $a$. But then $a$ and $c$ are not close so they are not the same (w.r.t. the program) even though $b$ and $c$ might have been closer to eachother than $a$ and $b$. We hence can have non-optimal identifications for vertices which, again, is a trade-off that is made for increased computational efficiency.

But note that if we were to start from vertex $b$, so check whether $b == a$ and not $a == b$, then $b$ and $a$ would be identified in the point $b$ and not $a$. As $b$ and $c$ are close as well, this would again be coincided in $b$ resulting in one and not two remaining vertices originating from the initial three.

This could lead to the case that, as the upper triangle approach is simply iterating through the vertices in the mesh beginning from the first while the sort approach may change the order for the sorting, we may in some rarer cases end up with a different amount of vertices. The deviating numbers of vertices could become an issue if we were to use both approach alongside eachother, which we are not. The upper triangle matrix was an early, naive approach to solve the problem for a small number of vertices where the $O(n^2)$ didn't affect the computational time significantly. It will be completely replaced by the sort approach as the used timsort is better on average and in the worst-case with $O(n\log n)$ and equal in the best-cast (linear). Also, timsort is a stable sorting algorithm that means the eevn if we had points that are exactly the same the order in which they are mentioned would be kept so there are no differences if applied at different times.